# Reddit Scrapper

In [5]:
import sys
sys.path.append('./lib/python3.7/site-packages')

In [6]:
!pwd

/Users/peerwoyzcechowski/Desktop/Privat/9 - Virtual machines/venv3.8


In [8]:
import sys
!{sys.executable} -m pip install praw

  Using cached praw-7.2.0-py3-none-any.whl (159 kB)
  Using cached prawcore-2.0.0-py3-none-any.whl (15 kB)
  Using cached websocket_client-0.58.0-py2.py3-none-any.whl (61 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)


In [16]:
import praw
import time
import numpy as np
import pandas as pd

## Reddit authentication

In [18]:
start_time = time.time()
reddit = praw.Reddit(
    user_agent="WebScrapper DataScienceProject",
    client_id="UetTsOSiJ7AJcQ",
    client_secret="_Q2D40oZ059J2Swsf_MNIJ-4e7L1zQ"
)

In [20]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit('wallstreetbets').hot(limit=10)
for post in hot_posts:
    print(post.title)

Daily Discussion Thread for March 04, 2021
WSB Rules - Please Read Before Posting
GME Megathread for March 04, 2021
Hard hitting investigative journalism & pultizer prize winning stuff right here
I combined all the top GME memes into 1 epic video
Lose enough money with Fidelity and they'll send you a letter perfect for framing
Casually threw a year's salary into GameStop
AMC to Open 35 More Movie Theaters Across Illinois Friday
GME IS UNSTOPPABLE! See you on the moon tards! 💎👐🚀🚀
It’s not worth it.


In [ ]:
posts = []
ml_subreddit = reddit.subreddit('MachineLearning')
for post in ml_subreddit.hot(limit=10):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
print(posts)

## Defining parameters

In [12]:
'''############################################################################'''
# The program parameters
subs = ['wallstreetbets', 'stocks', 'investing', 'stockmarket']     # sub-reddit to search
post_flairs = {'Daily Discussion', 'Weekend Discussion', 'Discussion','DD','News'}    # posts flairs to search || None flair is automatically considered
goodAuth = {'AutoModerator'}   # authors whom comments are allowed more than once
uniqueCmt = True                # allow one comment per author per symbol
ignoreAuthP = {'example'}       # authors to ignore for posts 
ignoreAuthC = {'example'}       # authors to ignore for comment 
upvoteRatio = 0.70         # upvote ratio for post to be considered, 0.70 = 70%
ups = 20       # define # of upvotes, post is considered if upvotes exceed this #
limit = 10      # define the limit, comments 'replace more' limit
upvotes = 2     # define # of upvotes, comment is considered if upvotes exceed this #
picks = 10     # define # of picks here, prints as "Top ## picks are:"
picks_ayz = 5   # define # of picks for sentiment analysis
'''############################################################################'''

posts, count, c_analyzed, tickers, titles, a_comments = 0, 0, 0, {}, [], {}
cmt_auth = {}

for sub in subs:
    subreddit = reddit.subreddit(sub)
    hot_python = subreddit.hot()    # sorting posts by hot
    # Extracting comments, symbols from subreddit
    for submission in hot_python:
        flair = submission.link_flair_text 
        author = submission.author.name         
        
        # checking: post upvote ratio # of upvotes, post flair, and author 
        if submission.upvote_ratio >= upvoteRatio and submission.ups > ups and (flair in post_flairs or flair is None) and author not in ignoreAuthP:   
            submission.comment_sort = 'new'     
            comments = submission.comments
            titles.append(submission.title)
            posts += 1
            submission.comments.replace_more(limit=limit)   
            for comment in comments:
                # try except for deleted account?
                try: auth = comment.author.name
                except: pass
                c_analyzed += 1
                
                # checking: comment upvotes and author
                if comment.score > upvotes and auth not in ignoreAuthC:      
                    split = comment.body.split(" ")
                    for word in split:
                        word = word.replace("$", "")        
                        # upper = ticker, length of ticker <= 5, excluded words,                     
                        if word.isupper() and len(word) <= 5 and word not in blacklist and word in us:
                            
                            # unique comments, try/except for key errors
                            if uniqueCmt and auth not in goodAuth:
                                try: 
                                    if auth in cmt_auth[word]: break
                                except: pass
                                
                            # counting tickers
                            if word in tickers:
                                tickers[word] += 1
                                a_comments[word].append(comment.body)
                                cmt_auth[word].append(auth)
                                count += 1
                            else:                               
                                tickers[word] = 1
                                cmt_auth[word] = [auth]
                                a_comments[word] = [comment.body]
                                count += 1    



NameError: name 'time' is not defined